In [14]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import openpyxl
import re
import unicodedata
import pandas as pd

In [3]:
respuestas_esp = pd.read_excel('Respuestas Playbook ESP.xlsx')

In [ ]:
drops_esp = ['Columna 11', 'Columna 3', '¿Te gustaría participar en la rifa de un PREMIO? (Toma 1 minuto más y tu participación es completamente anónima)',
       'Ingresa tu código único que incluya:\n- 8 letras (pueden repetirse)\n- 2 números (pueden repetirse)\n- 2 símbolos especiales como: #, @, !, %, &, etc.',
       '¿Te refirió alguien para contestar esta encuesta? Ingresa su código de referido']

respuestas_esp = respuestas_esp.drop(drops_esp, axis = 1)

respuestas_esp['non_fan'] = respuestas_esp['¿Con qué frecuencia ves fútbol femenino?'].apply(lambda x: 'No Fan' if x == 'Nunca' else 'Fan')

In [10]:
for c in respuestas_esp.columns:
    print(c)
    # print 5 rows of the column
    print("Head:")
    print(respuestas_esp[c].head())
    try:
        print("Unique:")
        print(respuestas_esp[c].unique()[0:10])
    except:
        pass
    print('--------------------------------')
    print('--------------------------------')

Marca temporal
Head:
0   2025-04-11 13:58:15.177
1   2025-04-11 17:35:34.294
2   2025-04-11 17:48:24.986
3   2025-04-15 08:01:09.663
4   2025-04-15 08:35:39.010
Name: Marca temporal, dtype: datetime64[ns]
Unique:
<DatetimeArray>
['2025-04-11 13:58:15.177000', '2025-04-11 17:35:34.294000',
 '2025-04-11 17:48:24.986000', '2025-04-15 08:01:09.663000',
 '2025-04-15 08:35:39.010000', '2025-04-15 08:38:29.889000',
 '2025-04-15 09:07:13.057000', '2025-04-15 09:30:26.967000',
 '2025-04-15 09:32:19.234000', '2025-04-15 10:55:14.477000']
Length: 10, dtype: datetime64[ns]
--------------------------------
--------------------------------
¿En qué país resides?
Head:
0     Mexico
1     Mexico
2    México 
3     México
4     México
Name: ¿En qué país resides?, dtype: object
Unique:
['Mexico' 'México ' 'México' 'mexico' 'Colombia' 'Ecuador' 'Perú'
 'Estados Unidos ' 'MÉXICO ' 'MEXICO ']
--------------------------------
--------------------------------
¿Cuál es tu rango de edad?
Head:
0    25-34
1    2

In [63]:
cols = ['¿Crees que el fútbol femenino debería recibir la misma inversión comercial que el masculino?',
        '¿Apoyarías o boicotearías una marca según su apoyo al fútbol femenino?',
        '¿Qué sientes cuando las marcas usan a deportistas femeninas en sus campañas o anuncios?',
        '¿Cómo crees que crecerá el fútbol femenino en tu país en los próximos 5 años?']

for c in cols:
    print(c)
    print(respuestas_esp[c].unique())
    print('--------------------------------')
    print('--------------------------------') 

¿Crees que el fútbol femenino debería recibir la misma inversión comercial que el masculino?
[nan 'Sí' 'No' 'No estoy seguro/a']
--------------------------------
--------------------------------
¿Apoyarías o boicotearías una marca según su apoyo al fútbol femenino?
[nan 'Boicotearía marcas que lo ignoran o se oponen'
 'Apoyaría marcas que invierten en fútbol femenino'
 'No cambiaría mi comportamiento']
--------------------------------
--------------------------------
¿Qué sientes cuando las marcas usan a deportistas femeninas en sus campañas o anuncios?
[nan 'Me gusta – me genera más confianza en la marca' 'Me inspira'
 'No lo noto' 'Me parece forzado o superficial']
--------------------------------
--------------------------------
¿Cómo crees que crecerá el fútbol femenino en tu país en los próximos 5 años?
[nan 'Crecerá significativamente' 'Crecerá lentamente'
 'Se mantendrá igual' 'No estoy seguro/a']
--------------------------------
--------------------------------


In [ ]:
col_sigue = "¿Sigues a equipos o jugadoras específicas en el fútbol femenino?"

def featurize_sigue(txt: str) -> pd.Series:
    """
    Featuriza las respuestas de la columna de sigue en una serie de dummies.

    Args:
        txt (str): La respuesta de la columna de sigue

    Returns:
        pd.Series: La serie de dummies
    """
    if pd.isna(txt):
        return pd.Series({"sigue_equipos": np.nan, "sigue_jugadoras": np.nan})
    t = _normalize_key(txt)
    if "ambos" in t:
        return pd.Series({"sigue_equipos": 1, "sigue_jugadoras": 1})
    if "equipos" in t:
        return pd.Series({"sigue_equipos": 1, "sigue_jugadoras": 0})
    if "jugadoras" in t:
        return pd.Series({"sigue_equipos": 0, "sigue_jugadoras": 1})
    if "no aplica" in t or t == "no":
        return pd.Series({"sigue_equipos": 0, "sigue_jugadoras": 0})
    # fallback
    return pd.Series({"sigue_equipos": np.nan, "sigue_jugadoras": np.nan})

sigue_feats = respuestas_esp[col_sigue].apply(featurize_sigue)
respuestas_esp = pd.concat([respuestas_esp, sigue_feats], axis=1)

In [ ]:
col_sigue = "¿Sigues a equipos o jugadoras específicas en el fútbol femenino?"

def featurize_sigue(txt: str) -> pd.Series:
    """
    Featuriza las respuestas de la columna de sigue en una serie de dummies.

    Args:
        txt (str): La respuesta de la columna de sigue

    Returns:
        pd.Series: La serie de dummies
    """
    if pd.isna(txt):
        return pd.Series({"sigue_equipos": np.nan, "sigue_jugadoras": np.nan})
    t = _normalize_key(txt)
    if "ambos" in t:
        return pd.Series({"sigue_equipos": 1, "sigue_jugadoras": 1})
    if "equipos" in t:
        return pd.Series({"sigue_equipos": 1, "sigue_jugadoras": 0})
    if "jugadoras" in t:
        return pd.Series({"sigue_equipos": 0, "sigue_jugadoras": 1})
    if "no aplica" in t or t == "no":
        return pd.Series({"sigue_equipos": 0, "sigue_jugadoras": 0})
    # fallback
    return pd.Series({"sigue_equipos": np.nan, "sigue_jugadoras": np.nan})

sigue_feats = respuestas_esp[col_sigue].apply(featurize_sigue)
respuestas_esp = pd.concat([respuestas_esp, sigue_feats], axis=1)

In [23]:
def _normalize_key(x: str) -> str:
    """
    a) strip espacios (extremos)
    b) elimina acentos
    c) deja solo letras y espacios
    d) lowercase y colapsa espacios

    Args:
        x (str): La cadena a normalizar

    Returns:
        str: La cadena normalizada
    """
    # limpiamos espacios del extremo
    x = str(x).strip()
    # eliminamos los acentos y caractéres especiales como tildes o diéresis o emojis
    # Ojo: los caractéres sin equivalencias se pierden; dudo que sea tema para nosotros tho
    x = unicodedata.normalize("NFKD", x).encode("ascii", "ignore").decode("ascii")
    # Cualquier caractér que no sea letra o espacio, lo reemplazamos por un espacio
    x = re.sub(r"[^a-zA-Z ]+", " ", x) 
    # colapsamos los espacios y convertimos a lowercase
    x = re.sub(r"\s+", " ", x).strip().lower() 
    return x

MAP = {
    # Muy casero pero es lo que tenemos y lo que funciona
    "mexico": "México",
    "mexco": "México",
    "mexuco": "México",
    "cdmx": "México",
    "ciudad de mexico": "México",
    "estados unidos": "Estados Unidos",
    "usa": "Estados Unidos",
    "colombia": "Colombia",
    "ecuador": "Ecuador",
    "peru": "Perú",
    "brasil": "Brasil",
    "argentina": "Argentina",
    "costa rica": "Costa Rica",
    "alemania": "Alemania",
    "dinamarca": "Dinamarca",
    "espana": "España",
    "canada": "Canadá",
    "paises bajos": "Países Bajos",
    "puerto rico": "Puerto Rico",
    "el salvador": "El Salvador",
    "guatemala": "Guatemala",
}

def homogeneizar_pais(x: str) -> str:
    """
    Normaliza el nombre de un país para que sea consistente con el diccionario MAP.

    Args:
        x (str): El nombre del país a normalizar

    Returns:
        str: El nombre del país normalizado
    """
    if pd.isna(x): # si el valor es NaN, no lo vamos a tocar
        return x
    # normalizamos el texto limpiando espacios, acentos, etc.
    key = _normalize_key(x)
    # si el texto normalizado está en el diccionario, lo devolvemos
    # si no, devolvemos el texto original sin espacios en los exteemos 
    return MAP.get(key, str(x).strip())

In [ ]:
def transforma_edad(df: pd.DataFrame, age_col: str) -> pd.DataFrame:
    """
    Transforma la columna de edad en una columna ordinal y una columna de punto medio.

    Args:
        df (pd.DataFrame): El DataFrame con la columna de edad
        age_col (str): El nombre de la columna de edad

    Returns:
        pd.DataFrame: El DataFrame con las columnas de edad ordinal y punto medio
    """

    out = df.copy()

    # Orden lógico para análisis/visualización y uso en modelado
    age_order = ["Menor de 18", "18-24", "25-34", "35-44", "45-54", "55+"]

    age_label_to_ordinal = {
        "Menor de 18": 0, "18-24": 1, "25-34": 2, "35-44": 3, "45-54": 4, "55+": 5
    }

    # Punto medio para analísis demográfico
    # En 18 pongo 16.0 porque es el punto medio entre 15 y 18 y está a 5 años de distancia de los otros grupos
    age_label_to_midpoint = {
        "Menor de 18": 16.5, "18-24": 21.0, "25-34": 30.0,
        "35-44": 40.0, "45-54": 50.0, "55+": 60.0
    }

    # Mapeo
    out["edad_ordinal"] = out[age_col].map(age_label_to_ordinal).astype("Int64")
    out["edad_midpoint"] = out[age_col].map(age_label_to_midpoint).astype("float64")

    return out
    
def transforma_genero(df: pd.DataFrame, gender_col: str) -> pd.DataFrame:
    """
    Transforma la columna de género en dummies.

    Args:
        df (pd.DataFrame): El DataFrame con la columna de género
        gender_col (str): El nombre de la columna de género

    Returns:
        pd.DataFrame: El DataFrame con las columnas de género dummies
    """
    out = df.copy()
    
    out["genero_categoria"] = pd.Categorical(gender_col, ordered=False)
    
    # Transformación en dummies del estilo genero_{genero}
    # OJO: se quedan las 3 categorías, hombre, mujer y otro.
    dummies_genero = pd.get_dummies(out["genero_categoria"], prefix="genero", dtype="uint8")
    out = pd.concat([out, dummies_genero], axis=1)
    
    return out

In [ ]:
def _slug(x: str) -> str:
    """
    Normaliza el nombre de una columna para que sea consistente con el diccionario MAP.

    Args:
        x (str): El nombre de la columna a normalizar

    Returns:
        str: El nombre de la columna normalizado
    """
    x = unicodedata.normalize("NFKD", str(x)).encode("ascii", "ignore").decode("ascii")
    x = re.sub(r"[^a-zA-Z0-9]+", "_", x)
    return x.strip("_").lower()



In [ ]:
rel_cols = ["fanatico", "atleta_amateur", "atleta_profesional", "trabajo_industria", "no_activo"]

def _ohe_relacion(txt: str) -> pd.Series:
    """
    Transforma la columna de relación con el deporte en dummies.

    Args:
        txt (str): El texto de la columna de relación con el deporte

    Returns:
        pd.Series: Las columnas de relación con el deporte dummies
    """
    out = {k: 0 for k in rel_cols}
    if pd.isna(txt): 
        return pd.Series(out)
    t = _normalize_key(txt)

    if "fanatic" in t: out["fanatico"] = 1
    if "amateur" in t: out["atleta_amateur"] = 1
    if "profesional" in t: out["atleta_profesional"] = 1
    if "industria" in t or "trabajo en la industria" in t: out["trabajo_industria"] = 1

    if "no sigo ni trabajo activamente" in t or re.search(r"\bno sigo\b", t):
        out["no_activo"] = 1

    return pd.Series(out)

col_rel = '¿Cuál es tu relación con el deporte? (Elige todas las que apliquen)'

respuestas_esp[[f"rel_{c}" for c in rel_cols]] = respuestas_esp[col_rel].apply(_ohe_relacion)

In [30]:
col_freq = "¿Con qué frecuencia ves fútbol femenino?"

FREQ_ORDER = [
    "Nunca",
    "Menos de una vez al mes",
    "Al menos dos veces al mes",
    "Semanalmente",
    "Veo cada partido que puedo",
]

FREQ_TO_ORD = {
    "Nunca": 0,
    "Menos de una vez al mes": 1,
    "Al menos dos veces al mes": 2,
    "Semanalmente": 3,
    "Veo cada partido que puedo": 4,
}

# Vectorizado, simple:
respuestas_esp["freq_ord"] = respuestas_esp[col_freq].map(FREQ_TO_ORD)

respuestas_esp["freq_cat"] = pd.Categorical(
    respuestas_esp[col_freq],
    categories=FREQ_ORDER,
    ordered=True
)

In [ ]:
col_canales = "¿A través de qué canales sigues los partidos de fútbol femenino en vivo? (Selecciona todos los que apliquen)"

def _normalize_key(x: str) -> str:
    x = str(x)
    x = unicodedata.normalize("NFKD", x).encode("ascii", "ignore").decode("ascii")
    x = re.sub(r"\s+", " ", x).strip().lower()
    return x

CANALES = [
    "tv_abierta",
    "tv_cable",
    "streaming",
    "redes_sociales",
    "radio",
    "estadio",
    "app_deportiva",
    "youtube",
]

ALIAS2CANAL = {
    # TV abierta / cable
    "tv abierta": "tv_abierta",
    "tv por cable": "tv_cable",
    "sky soccer": "tv_cable",
    "sky": "tv_cable",
    # Streaming / plataformas
    "servicio de streaming": "streaming",
    "streaming": "streaming",
    "youtube": "youtube",
    "dazn": "streaming", 
    "danz": "streaming", # aparece com o danz jaja
    # Redes / radio / estadio / app
    "redes sociales": "redes_sociales",
    "radio": "radio",
    "asistencia al estadio": "estadio",
    "aplicacion deportiva": "app_deportiva",
    # IPTV / Smart TV
    "iptv": "tv_cable",
    "samsung tv": "tv_cable",
}

NEGACIONES = ["no sigo los juegos en vivo", "no aplica"]
INDETERMINADO = ["donde lo pasen"]

def featurize_canales(txt: str) -> pd.Series:
    """
    Featuriza las respuestas de la columna de canales en una serie de dummies.

    Args:
        txt (str): La respuesta de la columna de canales

    Returns:
        pd.Series: La serie de dummies
    """
    out = {f"canal__{c}": 0 for c in CANALES}
    flags = {"canal__no_en_vivo": 0, "canal__indiferente": 0}

    if pd.isna(txt):
        return pd.Series({**out, **flags})

    t = _normalize_key(txt)

    if any(neg in t for neg in NEGACIONES):
        flags["canal__no_en_vivo"] = 1
        return pd.Series({**out, **flags})

    if any(ind in t for ind in INDETERMINADO):
        flags["canal__indiferente"] = 1

    for alias, canal in ALIAS2CANAL.items():
        if alias in t:
            out[f"canal__{canal}"] = 1

    return pd.Series({**out, **flags})

canales_feats = respuestas_esp[col_canales].apply(featurize_canales)
respuestas_esp = pd.concat([respuestas_esp, canales_feats], axis=1)

In [ ]:
col_redes = "¿En qué redes sociales sigues contenido de fútbol femenino? (Selecciona todas las que apliquen)"

REDES = ["instagram", "twitter_x", "facebook", "tiktok", "youtube"]

ALIAS2RED = {
    "instagram": "instagram",
    "twitter/x": "twitter_x",
    "twitter": "twitter_x",  # incluye "twitter/x, …"
    "facebook": "facebook",
    "tiktok": "tiktok",
    "youtube": "youtube",
    "you tube": "youtube",
}

NEGACIONES_RED = ["no lo sigo en redes sociales", "no aplica"]

def featurize_redes(txt: str) -> pd.Series:
    """
    Featuriza las respuestas de la columna de redes en una serie de dummies.

    Args:
        txt (str): La respuesta de la columna de redes

    out = {f"rs__{r}": 0 for r in REDES}
    """
    flags = {"rs__no_redes": 0, "rs__no_aplica": 0}
    if pd.isna(txt):
        return pd.Series({**out, **flags})

    t = _normalize_key(txt)

    if "no aplica" in t:
        flags["rs__no_aplica"] = 1
        return pd.Series({**out, **flags})
    if "no lo sigo en redes sociales" in t:
        flags["rs__no_redes"] = 1
        return pd.Series({**out, **flags})

    for alias, red in ALIAS2RED.items():
        if alias in t:
            out[f"rs__{red}"] = 1

    return pd.Series({**out, **flags})

redes_feats = respuestas_esp[col_redes].apply(featurize_redes)
respuestas_esp = pd.concat([respuestas_esp, redes_feats], axis=1)

In [50]:
col_tipo = "¿Qué tipo de contenido consumes más? (Selecciona todas las que apliquen)"

TIPOS = [
    "resumenes_highlights",
    "entrevistas",
    "estadisticas_analisis",
    "detras_camaras",
    "contenido_fans",
    "contenido_marca_patrocinado",
    "noticieros_profesionales",
    "noticieros_independientes",
    "contenido_club",
    "creadoras_contenido",
]

ALIAS2TIPO = {
    # Resúmenes
    "resumenes": "resumenes_highlights",
    "highlights": "resumenes_highlights",
    # Entrevistas
    "entrevistas a jugadoras": "entrevistas",
    "entrevistas a entrenadores": "entrevistas",
    "entrevistas": "entrevistas",
    # Estadísticas / análisis
    "estadisticas del juego": "estadisticas_analisis",
    "analisis tactico": "estadisticas_analisis",
    "analisis": "estadisticas_analisis",
    # Detrás de cámaras / vida de equipo
    "detras de camaras": "detras_camaras",
    "vida de equipo": "detras_camaras",
    # Fans / creadoras
    "contenido generado por fans": "contenido_creado_por_usuarios",
    "creadoras de contenido": "contenido_creado_por_usuarios",
    # Marca / patrocinado
    "contenido de marca": "contenido_marca_patrocinado",
    "patrocinado": "contenido_marca_patrocinado",
    # Noticieros
    "noticieros profesionales": "noticieros_profesionales",
    "noticieros independientes": "noticieros_independientes",
    # Club
    "contenido del club": "contenido_club",
}

NEGACIONES_TIPO = ["no aplica"]

def featurize_tipos(txt: str) -> pd.Series:
    """
    Featuriza las respuestas de la columna de tipos en una serie de dummies.

    Args:
        txt (str): La respuesta de la columna de tipos

    Returns:
        pd.Series: La serie de dummies
    """
    out = {f"cont__{t}": 0 for t in TIPOS}
    flags = {"cont__no_aplica": 0}
    if pd.isna(txt):
        return pd.Series({**out, **flags})

    t = _normalize_key(txt)

    if any(neg in t for neg in NEGACIONES_TIPO):
        flags["cont__no_aplica"] = 1
        return pd.Series({**out, **flags})

    for alias, tipo in ALIAS2TIPO.items():
        if alias in t:
            out[f"cont__{tipo}"] = 1

    return pd.Series({**out, **flags})

tipos_feats = respuestas_esp[col_tipo].apply(featurize_tipos)
respuestas_esp = pd.concat([respuestas_esp, tipos_feats], axis=1)

In [48]:
respuestas_esp['¿Sigues a equipos o jugadoras específicas en el fútbol femenino?'].unique()

array([nan, 'Ambos', 'No aplica', 'Sí – equipos específicos', 'No',
       'Sí – jugadoras específicas'], dtype=object)

In [ ]:
col_sigue = "¿Sigues a equipos o jugadoras específicas en el fútbol femenino?"

def featurize_sigue(txt: str) -> pd.Series:
    """
    Featuriza las respuestas de la columna de sigue en una serie de dummies.

    Args:
        txt (str): La respuesta de la columna de sigue

    Returns:
        pd.Series: La serie de dummies
    """
    if pd.isna(txt):
        return pd.Series({"sigue_equipos": np.nan, "sigue_jugadoras": np.nan})
    t = _normalize_key(txt)
    if "ambos" in t:
        return pd.Series({"sigue_equipos": 1, "sigue_jugadoras": 1})
    if "equipos" in t:
        return pd.Series({"sigue_equipos": 1, "sigue_jugadoras": 0})
    if "jugadoras" in t:
        return pd.Series({"sigue_equipos": 0, "sigue_jugadoras": 1})
    if "no aplica" in t or t == "no":
        return pd.Series({"sigue_equipos": 0, "sigue_jugadoras": 0})
    # fallback
    return pd.Series({"sigue_equipos": np.nan, "sigue_jugadoras": np.nan})

sigue_feats = respuestas_esp[col_sigue].apply(featurize_sigue)
respuestas_esp = pd.concat([respuestas_esp, sigue_feats], axis=1)

In [67]:
respuestas_esp['¿Has asistido alguna vez a un partido de fútbol femenino en vivo?'].unique()

array([nan, 'Sí, asisto con frecuencia', 'No, pero me gustaría',
       'Sí, una o dos veces máximo', 'No, y no me interesa', 'No aplica'],
      dtype=object)

In [ ]:
col_asist = "¿Has asistido alguna vez a un partido de fútbol femenino en vivo?"

def featurize_asistencia(txt: str) -> pd.Series:
    """
    Featuriza las respuestas de la columna de asistencia en una serie de dummies.

    Args:
        txt (str): La respuesta de la columna de asistencia

    Returns:
        pd.Series: La serie de dummies
    """
    if pd.isna(txt):
        return pd.Series({"asist_ord": np.nan, "ha_asistido": np.nan})

    t = _normalize_key(txt)
    if "con frecuencia" in t:
        k = 2
    elif "una o dos" in t or "una o dos veces" in t:
        k = 1
    elif "no aplica" in t:
        return pd.Series({"asist_ord": np.nan, "ha_asistido": 0})
    elif "no" in t:  # cubre "no, pero me gustaria" y "no, y no me interesa"
        k = 0
    else:
        k = np.nan
        
    ha = (k in (1, 2)) * 1 if not pd.isna(k) else np.nan
    return pd.Series({"asist_ord": k, "ha_asistido": ha})

asist_feats = respuestas_esp[col_asist].apply(featurize_asistencia)
respuestas_esp = pd.concat([respuestas_esp, asist_feats], axis=1)

In [65]:
col_perc = "¿Cómo cambia tu percepción de una marca al verla patrocinando fútbol femenino?"

PERCEP_MAP_NORM = {
    "Mucho menos favorable": -2,
    "Algo menos favorable": -1,
    "Sin cambio": 0,
    "Algo más favorable": 1,  
    "Mucho más favorable": 2,
}

# Simplemente convertirla a ordinal (ya despues si la normalizmaos o no, da igual)
def map_percepcion(x: str) -> int:
    """
    Mapea la percepción de una marca al verla patrocinando fútbol femenino a un valor ordinal.

    Args:
        x (str): La respuesta de la columna de percepción

    Returns:
        int: El valor ordinal de la percepción
    """
    if pd.isna(x): 
        return np.nan
    # normaliza "más"->"mas"
    return PERCEP_MAP_NORM.get(x, np.nan)

respuestas_esp["percep_patrocinio_ord"] = respuestas_esp[col_perc].apply(map_percepcion)

In [68]:
respuestas_esp[col_compra].unique()

array([nan, 'No', 'No estoy seguro/a', 'Sí', 'No aplica'], dtype=object)

In [69]:
col_compra = "¿Has comprado un producto o usado un servicio porque patrocinaba un equipo o atleta de deportes femeninos?"

COMPRA_MAP_NORM = {
    "Sí": 1,
    "No": -1,
    "No estoy seguro/a": 0,
    "No aplica": 0,
}

def map_compra(x: str) -> int:
    """
    Mapea la respuesta de la columna de compra a un valor ordinal.

    Args:
        x (str): La respuesta de la columna de compra

    Returns:
        int: El valor ordinal de la compra
    """
    if pd.isna(x): 
        return np.nan

    return COMPRA_MAP_NORM.get(x, np.nan)

respuestas_esp["compra_patrocinio_cat"] = respuestas_esp[col_compra].apply(map_compra).astype("category")
respuestas_esp["compra_influenciada"] = respuestas_esp["compra_patrocinio_cat"].map({"si": 1, "no": 0}).astype("Int64")

In [70]:
col_imp = "¿Qué tan importante es para ti que las marcas apoyen el deporte femenino?"

imp_num = pd.to_numeric(respuestas_esp[col_imp], errors="coerce").astype("Int64")
respuestas_esp["importancia_marcas"] = imp_num
respuestas_esp["importancia_marcas_cat"] = pd.Categorical(imp_num, categories=[1,2,3,4,5], ordered=True)

In [73]:
col_inv = "¿Crees que el fútbol femenino debería recibir la misma inversión comercial que el masculino?"

def map_inversion(x: str) -> pd.Series:
    """
    Mapea la respuesta de la columna de inversión a un valor ordinal.

    Args:
        x (str): La respuesta de la columna de inversión

    Returns:
        pd.Series: La serie de dummies
    """
    if pd.isna(x): 
        return pd.Series({"inversion_igual_ord": np.nan, "inversion_igual_cat": np.nan})
    t = _normalize_key(x)
    if t.startswith("si"):
        return pd.Series({"inversion_igual_ord": 1, "inversion_igual_cat": "si"})
    if t == "no":
        return pd.Series({"inversion_igual_ord": 0, "inversion_igual_cat": "no"})
    if "no estoy seguro" in t:
        return pd.Series({"inversion_igual_ord": -11, "inversion_igual_cat": "no_seguro"})
    return pd.Series({"inversion_igual_ord": np.nan, "inversion_igual_cat": np.nan})

inv_feats = respuestas_esp[col_inv].apply(map_inversion)
respuestas_esp = pd.concat([respuestas_esp, inv_feats], axis=1)
respuestas_esp["inversion_igual_cat"] = respuestas_esp["inversion_igual_cat"].astype("category")

dummies_inv = pd.get_dummies(respuestas_esp["inversion_igual_cat"], prefix="inversion_igual", dtype="Int64")
respuestas_esp = pd.concat([respuestas_esp, dummies_inv], axis=1)

In [75]:
col_act = "¿Apoyarías o boicotearías una marca según su apoyo al fútbol femenino?"

def map_actitud(x):
    if pd.isna(x):
        return pd.Series({"actitud_marca_ff_ord": np.nan, "actitud_marca_ff_cat": np.nan})
    t = _normalize_key(x)
    if "boicot" in t:
        return pd.Series({"actitud_marca_ff_ord": -1, "actitud_marca_ff_cat": "boicot"})
    if "no cambiaria" in t or "no cambia" in t:
        return pd.Series({"actitud_marca_ff_ord": 0, "actitud_marca_ff_cat": "no_cambia"})
    if "apoyar" in t or "apoyaria" in t:
        return pd.Series({"actitud_marca_ff_ord": 1, "actitud_marca_ff_cat": "apoyo"})
    return pd.Series({"actitud_marca_ff_ord": np.nan, "actitud_marca_ff_cat": np.nan})

act_feats = respuestas_esp[col_act].apply(map_actitud)
respuestas_esp = pd.concat([respuestas_esp, act_feats], axis=1)
respuestas_esp["actitud_marca_ff_cat"] = respuestas_esp["actitud_marca_ff_cat"].astype("category")

dummies_act = pd.get_dummies(respuestas_esp["actitud_marca_ff_cat"], prefix="actitud_marca_ff", dtype="Int64")
respuestas_esp = pd.concat([respuestas_esp, dummies_act], axis=1)

In [76]:
respuestas_esp['¿Qué sientes cuando las marcas usan a deportistas femeninas en sus campañas o anuncios?'].unique()

array([nan, 'Me gusta – me genera más confianza en la marca',
       'Me inspira', 'No lo noto', 'Me parece forzado o superficial'],
      dtype=object)

In [77]:
col_sent = "¿Qué sientes cuando las marcas usan a deportistas femeninas en sus campañas o anuncios?"

def map_sentimiento(x):
    if pd.isna(x): 
        return np.nan
    t = _normalize_key(x)
    if "forzado" in t or "superficial" in t:
        return -1
    if "no lo noto" in t:
        return 0
    if "inspira" in t:
        return 2
    if "me gusta" in t or "confianza" in t:
        return 1
    return np.nan

respuestas_esp["campanas_deportistas_ord"] = respuestas_esp[col_sent].apply(map_sentimiento)
